# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

import cv2
import csv
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
    raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


W1109 11:13:11.971504 13632 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'cloud_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'object-detection.pbtxt')

NUM_CLASSES = 1

## Download Model

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

W1109 11:13:12.566938 13632 deprecation_wrapper.py:119] From D:\UCIFSI\models\research\object_detection\utils\label_map_util.py:137: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [8]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
for img in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, img) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [9]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [10]:
def encoded_pixel(a):
    x1,y1,x2,y2 = a
    epa = []
    #print(x1,y2)
    h = y2-y1
    ep = (1400*x1)+y1
    maxa = (1400*(x2+1))
    while (ep<maxa):
        epa.append(ep)
        epa.append(h)
        ep = ep+1400
    return epa

In [11]:
img_path = 'D:/Dataset/understanding_cloud_organization/train_images3'
k = 0
for image_path in os.listdir(PATH_TO_TEST_IMAGES_DIR):
    print(image_path)
    if(k < 0):
        k += 1
    else:
        w = []
        s = []
        p = []
        q = []
        r = []
        image = Image.open(os.path.join(PATH_TO_TEST_IMAGES_DIR, image_path))
        # the array based representation of the image will be used later in order to prepare the
        # result image with boxes and labels on it.
        image_np = load_image_into_numpy_array(image)
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        # Actual detection.
        output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=8)
        db = []
        dc = []
        ds = []
        j = 1
        while(j < 5):
            for i in range(len(output_dict['detection_boxes'])):
                if(output_dict['detection_classes'][i] == j):
                    dc.append(output_dict['detection_classes'][i])
                    db.append(output_dict['detection_boxes'][i])
                    ds.append(output_dict['detection_scores'][i])
                    j += 1
                    #print(j)
                    break

        # Label 1
        t = []
        if(ds[0] > 0.50):
            a = int(db[0][0] * 2100)
            b = int(db[0][1] * 1400)
            c = int(db[0][2] * 2100)
            d = int(db[0][3] * 1400)
            t = encoded_pixel([a, b, c, d])
        name1 = image_path + '_Fish' 
        if(ds[0] > 0.50):
            w.append([name1, t])
        else:
            w.append([name1])

        # Label 2
        t = []
        if(ds[1] > 0.50):
            a = int(db[1][0] * 2100)
            b = int(db[1][1] * 1400)
            c = int(db[1][2] * 2100)
            d = int(db[1][3] * 1400)
            t = encoded_pixel([a, b, c, d])
        name2 = image_path + '_Flower'

        if(ds[1] < 0.50):
            w.append([name2])
        else:
            w.append([name2, t])

        # Label 3
        t = []
        if(ds[2] > 0.50):
            a = int(db[2][0] * 2100)
            b = int(db[2][1] * 1400)
            c = int(db[2][2] * 2100)
            d = int(db[2][3] * 1400)
            t = encoded_pixel([a, b, c, d])
        name3 = image_path + '_Gravel'
        if(ds[2] < 0.50):
            w.append([name3])
        else:
            w.append([name3, t])

        # Label 4

        if(ds[3] > 0.50):
            a = int(db[3][0] * 2100)
            b = int(db[3][1] * 1400)
            c = int(db[3][2] * 2100)
            d = int(db[3][3] * 1400)
            t = encoded_pixel([a, b, c, d])
        name4 = image_path + '_Sugar'

        if(ds[3] < 0.50):
            w.append([name4])
        else:
            w.append([name4, t])
        print(k)
        k += 1
        fields = ['Image_Label', 'EncodedPixels']
        filename = 'Sample_Submission1.csv'
        with open(filename, 'a+') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerows(w)

002f507.jpg
0
0035ae9.jpg
1
0038327.jpg
2
004f759.jpg
3
005ba08.jpg
4
006440a.jpg
5
006f61b.jpg
6
0078e5a.jpg
7
008fc8a.jpg
8
0096937.jpg
9
00a47f4.jpg
10
00a7177.jpg
11
00af77b.jpg
12
00c3256.jpg
13
00cb435.jpg
14
00d2a33.jpg
15
00d8e50.jpg
16
00db8b3.jpg
17
00dbd3c.jpg
18
00f7c63.jpg
19
0110417.jpg
20
0111e06.jpg
21
01505ce.jpg
22
0167118.jpg
23
016eec3.jpg
24
0193c10.jpg
25
01d8c02.jpg
26
01e155a.jpg
27
01e507a.jpg
28
01fff14.jpg
29
0202748.jpg
30
021f14d.jpg
31
022feb6.jpg
32
023ad21.jpg
33
024fe4a.jpg
34
02680e5.jpg
35
027df0e.jpg
36
02903db.jpg
37
02942bd.jpg
38
029d276.jpg
39
02a1a4f.jpg
40
02bd330.jpg
41
02d0918.jpg
42
02d692f.jpg
43
030e141.jpg
44
0312290.jpg
45
031bdd3.jpg
46
032518b.jpg
47
0327e1f.jpg
48
0332698.jpg
49
034e0ea.jpg
50
0397495.jpg
51
03bfe76.jpg
52
03c28bc.jpg
53
03e8318.jpg
54
03ec713.jpg
55
04022ad.jpg
56
0403d7b.jpg
57
04080dd.jpg
58
041b654.jpg
59
041eec9.jpg
60
042c901.jpg
61
04326a2.jpg
62
043bd90.jpg
63
043d7a2.jpg
64
044819a.jpg
65
0455035.jpg
66
0460c

519
20fa598.jpg
520
2119f7f.jpg
521
211c816.jpg
522
21632dd.jpg
523
2178dcf.jpg
524
2180ee4.jpg
525
219c917.jpg
526
21a8ab7.jpg
527
21c2431.jpg
528
21c8b7d.jpg
529
21d9337.jpg
530
2233581.jpg
531
223564f.jpg
532
2246e8a.jpg
533
225c23d.jpg
534
2267a21.jpg
535
226ce7c.jpg
536
2282dd8.jpg
537
22839a8.jpg
538
22908a4.jpg
539
229ce1a.jpg
540
22aceaa.jpg
541
22ad54b.jpg
542
22e7cc9.jpg
543
22e9c91.jpg
544
230437f.jpg
545
2308423.jpg
546
230c8c7.jpg
547
23253e3.jpg
548
2327b00.jpg
549
236f4fc.jpg
550
2371e39.jpg
551
2373f56.jpg
552
2379bbb.jpg
553
2393b27.jpg
554
23a937a.jpg
555
23c4d63.jpg
556
23c8280.jpg
557
23d12b2.jpg
558
23e104f.jpg
559
23f67c1.jpg
560
2414b4a.jpg
561
244c0f9.jpg
562
24511de.jpg
563
245a60c.jpg
564
2465c38.jpg
565
2472d3f.jpg
566
247d7a7.jpg
567
24cfbc3.jpg
568
2504a79.jpg
569
25590ac.jpg
570
2562353.jpg
571
2577dfe.jpg
572
25880b6.jpg
573
258bcf8.jpg
574
25929c2.jpg
575
259569b.jpg
576
25998d0.jpg
577
25c075b.jpg
578
25c5403.jpg
579
25d4456.jpg
580
25e4704.jpg
581
25e5

1030
464bbbc.jpg
1031
4663103.jpg
1032
4676ff1.jpg
1033
467c45c.jpg
1034
467f2d9.jpg
1035
4684ee5.jpg
1036
46941bb.jpg
1037
46d1d6a.jpg
1038
46e95f1.jpg
1039
4720c55.jpg
1040
4735d01.jpg
1041
4792651.jpg
1042
4792a7e.jpg
1043
479acdf.jpg
1044
47c8b59.jpg
1045
47d1355.jpg
1046
47e28e5.jpg
1047
47eaabe.jpg
1048
480abfa.jpg
1049
480ac4b.jpg
1050
4815724.jpg
1051
4825694.jpg
1052
4831ffe.jpg
1053
4853c7a.jpg
1054
485860c.jpg
1055
485877b.jpg
1056
4892d82.jpg
1057
48a3a17.jpg
1058
48c2591.jpg
1059
48cfbcf.jpg
1060
48d227a.jpg
1061
48e7748.jpg
1062
491b064.jpg
1063
493e5cc.jpg
1064
4943a64.jpg
1065
494e6be.jpg
1066
4953c10.jpg
1067
497332c.jpg
1068
4988d80.jpg
1069
4990058.jpg
1070
4990da1.jpg
1071
49a89af.jpg
1072
49b416a.jpg
1073
49c1387.jpg
1074
49d911a.jpg
1075
49e0f20.jpg
1076
4a0883f.jpg
1077
4a1ae5e.jpg
1078
4a1d59c.jpg
1079
4a38465.jpg
1080
4a3fca8.jpg
1081
4a5f182.jpg
1082
4a70485.jpg
1083
4a73cae.jpg
1084
4a77ca2.jpg
1085
4a8f4b7.jpg
1086
4aa11ad.jpg
1087
4ab65e7.jpg
1088
4ac6d3e.j

1512
66ceff6.jpg
1513
66e236e.jpg
1514
66e5939.jpg
1515
67566d6.jpg
1516
6770f2c.jpg
1517
6782693.jpg
1518
679a0d3.jpg
1519
679ee38.jpg
1520
67a6e8e.jpg
1521
67cb8db.jpg
1522
67d2fdf.jpg
1523
6828274.jpg
1524
68292d8.jpg
1525
68729b6.jpg
1526
68773cd.jpg
1527
687df45.jpg
1528
68860c8.jpg
1529
68b4159.jpg
1530
68c12d8.jpg
1531
68c221a.jpg
1532
68caccd.jpg
1533
68e6a87.jpg
1534
68e879e.jpg
1535
68f9bfa.jpg
1536
68ff3dd.jpg
1537
690f35c.jpg
1538
691bf7b.jpg
1539
691dfce.jpg
1540
692de93.jpg
1541
6936b4c.jpg
1542
6970e13.jpg
1543
698b13b.jpg
1544
6995ee0.jpg
1545
69a5c46.jpg
1546
69b497a.jpg
1547
69c04d8.jpg
1548
69c1ff3.jpg
1549
69c8fb4.jpg
1550
69d66ae.jpg
1551
6a0a646.jpg
1552
6a0deb9.jpg
1553
6a0efbb.jpg
1554
6a10abc.jpg
1555
6a1ae4f.jpg
1556
6a350fd.jpg
1557
6a48047.jpg
1558
6a571b1.jpg
1559
6a57c47.jpg
1560
6a629b9.jpg
1561
6a63dd9.jpg
1562
6aac900.jpg
1563
6aad364.jpg
1564
6ad35bf.jpg
1565
6ae7aa1.jpg
1566
6ae833f.jpg
1567
6af215f.jpg
1568
6b31e9b.jpg
1569
6b49b68.jpg
1570
6b5a025.j

1994
8b5dcc2.jpg
1995
8b65e62.jpg
1996
8b700ce.jpg
1997
8b73004.jpg
1998
8b7405c.jpg
1999
8b7ac29.jpg
2000
8b7d91e.jpg
2001
8b9304f.jpg
2002
8b952a6.jpg
2003
8b96a9a.jpg
2004
8b9b65a.jpg
2005
8ba6c10.jpg
2006
8bab421.jpg
2007
8bad299.jpg
2008
8bc82b5.jpg
2009
8bde7dd.jpg
2010
8bec556.jpg
2011
8c03e02.jpg
2012
8c4d1aa.jpg
2013
8c5937e.jpg
2014
8c6f583.jpg
2015
8cc89b6.jpg
2016
8cd0c71.jpg
2017
8cd8aa3.jpg
2018
8cedab9.jpg
2019
8d02e7c.jpg
2020
8d093d6.jpg
2021
8d15ede.jpg
2022
8d35f06.jpg
2023
8d43af3.jpg
2024
8d467d0.jpg
2025
8d52a7f.jpg
2026
8d564e5.jpg
2027
8d6beb8.jpg
2028
8d7d0c5.jpg
2029
8d87c7b.jpg
2030
8d8b0e7.jpg
2031
8d9f6f5.jpg
2032
8dc5112.jpg
2033
8ddff89.jpg
2034
8de73e4.jpg
2035
8e0dd0d.jpg
2036
8e24869.jpg
2037
8e31a0d.jpg
2038
8e4332c.jpg
2039
8e48779.jpg
2040
8e4880d.jpg
2041
8e4af27.jpg
2042
8e50293.jpg
2043
8e583cd.jpg
2044
8e67234.jpg
2045
8e7bad0.jpg
2046
8e86e36.jpg
2047
8e8f768.jpg
2048
8e92d2f.jpg
2049
8ea532b.jpg
2050
8eaaf5f.jpg
2051
8eb4a65.jpg
2052
8ecaaf4.j

2476
aeef245.jpg
2477
aefe2c5.jpg
2478
af04835.jpg
2479
af04ed3.jpg
2480
af0fd17.jpg
2481
af156be.jpg
2482
af16863.jpg
2483
af34d83.jpg
2484
af40920.jpg
2485
af4b3a7.jpg
2486
af69ed7.jpg
2487
af75f46.jpg
2488
af9e467.jpg
2489
afcea2f.jpg
2490
afe6444.jpg
2491
afeaabc.jpg
2492
b01b7dc.jpg
2493
b020d9e.jpg
2494
b048858.jpg
2495
b056133.jpg
2496
b05824f.jpg
2497
b06b4e1.jpg
2498
b09274d.jpg
2499
b0928ca.jpg
2500
b09441b.jpg
2501
b0b4d05.jpg
2502
b0b75f6.jpg
2503
b0b9685.jpg
2504
b0bdaff.jpg
2505
b0f2ab0.jpg
2506
b0fdffe.jpg
2507
b10c9f8.jpg
2508
b117feb.jpg
2509
b12acc9.jpg
2510
b1500df.jpg
2511
b15a8a3.jpg
2512
b16cf96.jpg
2513
b1771a4.jpg
2514
b186be9.jpg
2515
b18959d.jpg
2516
b18b09a.jpg
2517
b19620f.jpg
2518
b1b2a30.jpg
2519
b1bc0ea.jpg
2520
b1c798c.jpg
2521
b1cfac2.jpg
2522
b1f566f.jpg
2523
b1fafcb.jpg
2524
b2014b1.jpg
2525
b2032e6.jpg
2526
b21f6f7.jpg
2527
b22382f.jpg
2528
b23b305.jpg
2529
b23d3ed.jpg
2530
b24d251.jpg
2531
b24d4f3.jpg
2532
b2646b4.jpg
2533
b299984.jpg
2534
b2aed33.j

2958
d0a653c.jpg
2959
d0b9728.jpg
2960
d0d4e51.jpg
2961
d0d9d5f.jpg
2962
d0da6f1.jpg
2963
d0de5fd.jpg
2964
d0e3afb.jpg
2965
d0eb600.jpg
2966
d0f8d8d.jpg
2967
d117d5f.jpg
2968
d129060.jpg
2969
d1378d2.jpg
2970
d14ac5d.jpg
2971
d163fcc.jpg
2972
d16b147.jpg
2973
d18f8d8.jpg
2974
d1b038a.jpg
2975
d1c23a6.jpg
2976
d1c5119.jpg
2977
d1c900e.jpg
2978
d1d6a4c.jpg
2979
d1dc8e6.jpg
2980
d1e4077.jpg
2981
d1fba78.jpg
2982
d210cce.jpg
2983
d219e77.jpg
2984
d238a94.jpg
2985
d24a663.jpg
2986
d24f168.jpg
2987
d251d80.jpg
2988
d25395d.jpg
2989
d269328.jpg
2990
d26e8dd.jpg
2991
d271ad2.jpg
2992
d27e83e.jpg
2993
d28dc47.jpg
2994
d2a9756.jpg
2995
d2e6ea5.jpg
2996
d305fa6.jpg
2997
d306f03.jpg
2998
d3175ea.jpg
2999
d33574c.jpg
3000
d357cdd.jpg
3001
d35b482.jpg
3002
d3620d2.jpg
3003
d37623d.jpg
3004
d37f996.jpg
3005
d390108.jpg
3006
d39e266.jpg
3007
d3b14fd.jpg
3008
d3c6963.jpg
3009
d3d9be8.jpg
3010
d3e2b2d.jpg
3011
d3e4e08.jpg
3012
d3f716d.jpg
3013
d4125fe.jpg
3014
d430a71.jpg
3015
d43b08f.jpg
3016
d4475b4.j

3440
f0bb2a6.jpg
3441
f0c8bc7.jpg
3442
f0d1b39.jpg
3443
f0f0869.jpg
3444
f187f1b.jpg
3445
f1c71f9.jpg
3446
f2074c1.jpg
3447
f20e641.jpg
3448
f22bf2a.jpg
3449
f231e36.jpg
3450
f25f1a5.jpg
3451
f2659f1.jpg
3452
f26d352.jpg
3453
f28c0b6.jpg
3454
f2be4dd.jpg
3455
f2c8e1d.jpg
3456
f2e02f9.jpg
3457
f318474.jpg
3458
f31e309.jpg
3459
f32c701.jpg
3460
f338d43.jpg
3461
f33bc65.jpg
3462
f3489cb.jpg
3463
f34f4c1.jpg
3464
f37d9a3.jpg
3465
f38ef1d.jpg
3466
f3b5784.jpg
3467
f3d5dc8.jpg
3468
f3eba13.jpg
3469
f3f8737.jpg
3470
f40520e.jpg
3471
f4192b5.jpg
3472
f41a2af.jpg
3473
f41fe82.jpg
3474
f42057b.jpg
3475
f424f38.jpg
3476
f456d2d.jpg
3477
f45af29.jpg
3478
f468316.jpg
3479
f46c455.jpg
3480
f46f347.jpg
3481
f4cc09e.jpg
3482
f4ce36d.jpg
3483
f4e157f.jpg
3484
f4e1f15.jpg
3485
f4e2849.jpg
3486
f50af7a.jpg
3487
f51db70.jpg
3488
f524509.jpg
3489
f52b132.jpg
3490
f53d65f.jpg
3491
f54c18f.jpg
3492
f56a0a8.jpg
3493
f56b158.jpg
3494
f595d0b.jpg
3495
f59ba39.jpg
3496
f5ab036.jpg
3497
f5aed4a.jpg
3498
f5b8a45.j